In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pylab as plt
import numpy as np
import torch
import swyft.lightning as sl

In [3]:
class Simulator(sl.Simulator):
    def __init__(self, bounds = None):
        super().__init__()
        self.on_after_forward = sl.to_numpy32
        
    def forward(self, trace):
        z = trace.sample('z', np.random.rand, 3)
        x = trace.sample('x', lambda z: z + np.random.randn(3)*0.02, z)

In [4]:
class Network(sl.SwyftModule):
    def __init__(self, dropout = 0.1, lr = 1e-4):
        super().__init__()
        self.classifier = sl.RatioEstimatorMLP1d(3, 3, hidden_features = 256, dropout = self.hparams.dropout)
        
    def forward(self, x, z):
        x = x['x']
        z = z['z']
        ratios_z = self.classifier(x, z)
        return dict(z = ratios_z)

In [5]:
simulator = Simulator()
samples = simulator.sample(1000).to_numpy()

100%|██████████| 1000/1000 [00:00<00:00, 31347.56it/s]


In [6]:
datamodule = sl.SwyftDataModule(store = samples, batch_size = 128)

Deprecation warning: Use dataloaders directly rathe than this data module for transparency.


In [7]:
for lr in [1e-1, 1e-2, 1e-3, 1e-4]:
    network = Network(dropout = 0.2, lr = lr)
    trainer = sl.SwyftTrainer(accelerator = 'gpu', gpus=1, max_epochs = 100, **sl.tensorboard_config(save_dir = './lightning_logs2', name = '01-minimal-hparams', version=None))
    trainer.fit(network, datamodule)
    trainer.test(network, datamodule, ckpt_path = 'best')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params
---------------------------------------------------
0 | classifier | RatioEstimatorMLP1d | 800 K 
---------------------------------------------------
800 K     Trainable params
0         Non-trainable params
800 K     Total params
3.201     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  78%|███████▊  | 7/9 [00:00<00:00, 64.15it/s, loss=247, v_num=8] 
Validating: 0it [00:00, ?it/s]
Epoch 1:  78%|███████▊  | 7/9 [00:00<00:00, 74.74it/s, loss=149, v_num=8, val_loss=1.12e+3]
Validating: 0it [00:00, ?it/s]
Epoch 2:  78%|███████▊  | 7/9 [00:00<00:00, 74.24it/s, loss=108, v_num=8, val_loss=62.60]
Validating: 0it [00:00, ?it/s]
Epoch 3:  78%|███████▊  | 7/9 [00:00<00:00, 70.26it/s, loss=17.4, v_num=8, val_loss=10.10]
Validating: 0it [00:00, ?it/s]
Epoch 4:  78%|███████▊  | 7/9 [00:00<00:00, 75.52it/s, loss=5.6, v_num=8, val_loss=4.590] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  78%|███████▊  | 7/9 [00:00<00:00, 68.78it/s, loss=3.55, v_num=8, val_loss=3.720]
Validating: 0it [00:00, ?it/s]
Epoch 6:  78%|███████▊  | 7/9 [00:00<00:00, 73.30it/s, loss=2.97, v_num=8, val_loss=2.890]
Validating: 0it [00:00, ?it/s]
Epoch 7:  78%|███████▊  | 7/9 [00:00<00:00, 72.64it/s, loss=2.5, v_num=8, val_loss=2.430] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  78%|███████▊  | 7/9 [00:00<00:

...

Testing: 0it [00:00, ?it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'hp/JS-div': 1.1405411958694458, 'hp/KL-div': -7.927431583404541}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 8/8 [00:00<00:00, 135.54it/s]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params
---------------------------------------------------
0 | classifier | RatioEstimatorMLP1d | 800 K 
---------------------------------------------------
800 K     Trainable params
0         Non-trainable params
800 K     Total params
3.201     Total estimated model params size (MB)


Epoch 0:  78%|███████▊  | 7/9 [00:00<00:00, 82.09it/s, loss=4.02, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 1:  89%|████████▉ | 8/9 [00:00<00:00, 79.87it/s, loss=3.63, v_num=11, val_loss=4.160]
Validating: 0it [00:00, ?it/s]
Epoch 2:  78%|███████▊  | 7/9 [00:00<00:00, 78.13it/s, loss=3.21, v_num=11, val_loss=4.160]
Validating: 0it [00:00, ?it/s]
Epoch 3:  78%|███████▊  | 7/9 [00:00<00:00, 69.31it/s, loss=2.52, v_num=11, val_loss=4.140]
Validating: 0it [00:00, ?it/s]
Epoch 4:  78%|███████▊  | 7/9 [00:00<00:00, 68.61it/s, loss=1.98, v_num=11, val_loss=4.090]
Validating: 0it [00:00, ?it/s]
Epoch 5:  78%|███████▊  | 7/9 [00:00<00:00, 70.14it/s, loss=1.61, v_num=11, val_loss=3.890]
Validating: 0it [00:00, ?it/s]
Epoch 6:  78%|███████▊  | 7/9 [00:00<00:00, 76.84it/s, loss=1.38, v_num=11, val_loss=3.340]
Validating: 0it [00:00, ?it/s]
Epoch 7:  78%|███████▊  | 7/9 [00:00<00:00, 66.49it/s, loss=1.26, v_num=11, val_loss=2.540]
Validating: 0it [00:00, ?it/s]
Epoch 8:  78%|███████▊  | 7/9 [0

Restoring states from the checkpoint path at ./lightning_logs2/01-minimal-hparams/version_11/checkpoints/epoch=10-step=76.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ./lightning_logs2/01-minimal-hparams/version_11/checkpoints/epoch=10-step=76.ckpt


Testing:  88%|████████▊ | 7/8 [00:00<00:00, 64.67it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'hp/JS-div': 1.1909857988357544, 'hp/KL-div': -7.024380683898926}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 8/8 [00:00<00:00, 61.59it/s]
